In [2]:
%%writefile nlp.py

import nltk
import os
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from tika import parser
import string
import utils

# ps = nltk.PorterStemmer()
sw = set(stopwords.words('english'))

def preprocess(text):
    punct = "".join(string.punctuation)
    text = "".join([char.lower() for char in text if char not in punct])
    tokens = word_tokenize(text)
    # remove all tokens that are not alphabetic
    wordsisalpha = [word for word in tokens if word.isalpha()]
    #stemmed = [ps.stem(word) for word in wordsisalpha]
    final_token = [word for word in wordsisalpha if word not in sw]
    word_count = Counter(final_token)
    return word_count

def cosSimilarity(x, y):
    dot_product = np.dot(x,y)
    normx = np.linalg.norm(x)
    normy = np.linalg.norm(y)
    similarity = dot_product / (normx * normy)
    return similarity

def Similarity(dict1, dict2):
    words_list = []
    for key in dict1:
        words_list.append(key)
    for key in dict2:
        words_list.append(key)
    list_size = len(words_list)
    
    v1 = np.zeros(list_size, dtype= np.int)
    v2 = np.zeros(list_size, dtype= np.int)
    
    i = 0
    for (key) in words_list:
        v1[i] = dict1.get(key,0)
        v2[i] = dict2.get(key,0)
        i = i+1
    return cosSimilarity(v1, v2)

def clean(text): 
    text = text.lower()
    text = text.replace('\n', '')
    text = text.replace('_', '')
    text = text.replace('●', '')
    text = text.replace('•', '')
    return text
    
    
    

Overwriting nlp.py


In [90]:
%%writefile sklearn_cosine.py


def sk_cos_similarity(text_list):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(text_list)
    matchPercentage=cosine_similarity(count_matrix)[0][1]*100
    matchPercentage= round(matchPercentage, 2)
    return sk_cos_similarity




Overwriting sklearn_cosine.py


In [120]:
%%writefile jams.py



import utils

from nlp import preprocess, Similarity, clean

# from spacy_similarity import spacy_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# from sklearn_cosine import sk_cos_similarity
import streamlit as st 
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy
import spacy_streamlit

hide_streamlit_style = """
            <style>
            #MainMenu {visibility: hidden;}
            footer {visibility: hidden;}
            </style>
            """
st.markdown(hide_streamlit_style, unsafe_allow_html=True)
    


def_job = "Paste your job description here."
def_resume = "Paste your resume here."

st.sidebar.title('Which Similarity')
select_similarity = st.sidebar.selectbox('Select one',
                                         ['Home Brew Cosine', 'Spacy Similairty','Sklearn Cosine', 'BERT Cosine'])
st.title('Resume Match')

job = st.text_area('Paste your Job Description', def_job)
resume = st.text_area('Paste your Resume', def_resume)

nlp=spacy.load('en_core_web_md')

doc1 = nlp(job)
doc2 = nlp(resume)


### 'Home Brew Cosine'

pp_job = preprocess(job)
pp_resume = preprocess(resume)


matchPercentage = np.round((Similarity(pp_job, pp_resume)*100),2)


### Cosine Similarity
    
sk_job = clean(job)
sk_resume = clean(resume)

text_list = [sk_job, sk_resume]
cv = CountVectorizer()
count_matrix = cv.fit_transform(text_list)
matchPercentage_sk=cosine_similarity(count_matrix)[0][1]*100
matchPercentage_sk= round(matchPercentage_sk, 2)

### Spacy built in Similarity
matchPercentage_spacy= round(doc1.similarity(doc2),2)*100


if select_similarity == "Home Brew Cosine":
    if len(job) != 0 and len(resume) != 0:
        st.write("Your Resume matched", matchPercentage, '% with the job description')    
if select_similarity == "Sklearn Cosine":
    if len(job) != 0 and len(resume) != 0:
        st.write("Your Resume matched", matchPercentage_sk, '% with the job description')
if select_similarity=='Spacy Similarity':
    
    if len(job) != 0 and len(resume) != 0:
        st.write("Your Resume matched", matchPercentage_spacy, '% with the job description')
       

Overwriting jams.py


In [100]:
%%writefile cosine.py

import utils
import streamlit as st 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

job = st.text_area("This is a job description.", 'Paste your job description here.')
resume = st.text_area("This is a resume.", 'Paste your resume here')

job = job.lower()
job = job.replace('\n', '')
job = job.replace('_', '')

resume = resume.lower()
resume = resume.replace('\n', '')
resume = resume.replace('_', '')

text_list = [job, resume]

    
cv = CountVectorizer()
count_matrix = cv.fit_transform(text_list)
matchPercentage=cosine_similarity(count_matrix)[0][1]*100
matchPercentage= round(matchPercentage, 2)

st.write(matchPercentage)
    

Overwriting cosine.py


In [115]:
%%writefile spacytest.py

import streamlit as st 
import spacy
import spacy_streamlit

nlp=spacy.load('en_core_web_md')


job = st.text_area("This is a job description.", 'Paste your job description here.')
resume = st.text_area("This is a resume.", 'Paste your resume here')

job = job.lower()
job = job.replace('\n', '')
job = job.replace('_', '')

resume = resume.lower()
resume = resume.replace('\n', '')
resume = resume.replace('_', '')

### Spacy built in Similarity
doc1 = nlp(job)
doc2 = nlp(resume)
matchPercentage_spacy= round(doc1.similarity(doc2),2)*100

st.write("Your Resume matched", matchPercentage_spacy, '% with the job description')


Overwriting spacytest.py
